In [1]:
from bs4 import BeautifulSoup
import csv
from getpass import getpass
from msedge.selenium_tools import Edge,EdgeOptions
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import     NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select, WebDriverWait
from tqdm import tqdm
from time import sleep

In [46]:
options = EdgeOptions()
options.use_chromium = True
driver = Edge(options=options)

def access_script(driver, select):
    driver.find_element_by_link_text('Comparativos').click()
    driver.find_element_by_link_text('Quantitativos').click()
    driver.find_element_by_link_text('Mecenato').click()
    driver.find_element_by_link_text(select).click()
    driver.find_element_by_link_text('Por ano, região, uf, área e segmento cultural').click()
    element1 = driver.find_element_by_xpath('//img[@alt="Ministério da Cultura"]')
    hov1 = ActionChains(driver).move_to_element(element1)
    hov1.perform()
    
def extract_data(driver):
    frame = driver.find_element_by_name('Salicnet_iframe')
    driver.switch_to.frame(frame)
    
    wait = WebDriverWait(driver, 60)
    wait.until(EC.visibility_of_element_located((By.XPATH, '//input[@name="sub_form"]')))

    driver.execute_script('document.getElementById("sub_formb").click()')
    
    #Pegar a tabela que contém os dados desejados
    html=driver.page_source
    soup=BeautifulSoup(html,'html.parser')
    tables=soup.findAll("table")
    driver.switch_to.default_content()
    return tables[5]

def table_to_dataframe(data, desc):
    table=pd.read_html(str(data), header = 0)
    
    #dataframe da tabela com os dados
    df = table[0]
    
    #Criando uma couluna para identificar os dados de projetos já apresentados
    df['Tipo'] = desc
    
    #Substituindo todos os NaN por 0
    df = df.fillna(0)
    
    return df

#Ir para tela inicial 
driver.get('http://sistemas.cultura.gov.br/salicnet/Salicnet/Salicnet.php')

#Pegar os dados de projetos já apresentados
access_script(driver, 'De projetos apresentados')

#Pegar a tabela que contém os dados
data1 = extract_data(driver)

df_apresentados = table_to_dataframe(data1, "Apresentados")

#Pegar os dados de projetos já aprovados
access_script(driver, 'De projetos aprovados')

#Pegar a tabela que contém os dados
data2 = extract_data(driver)

df_aprovados = table_to_dataframe(data2, "Aprovados")


In [109]:
df_concated = pd.concat([df_aprovados,df_apresentados],axis = 0)

In [167]:
newHeader = {}

for i in df_aprovados.columns[1:31]:
    newHeader[i] = i.replace('.','')


In [156]:
df_concated.rename(columns = newHeader, inplace = True)

In [159]:
df_concated.to_csv("projetos_lei_rouanet.csv")

 ### Passo a passo

In [ ]:
#Primeiro Criamos uma instância do web driver
options = EdgeOptions()
options.use_chromium = True
driver = Edge(options=options)

In [ ]:
#Ir para tela inicial 
driver.get('http://sistemas.cultura.gov.br/salicnet/Salicnet/Salicnet.php')

In [ ]:
#Nevagar na página até chegar na página com relatórios desejados
driver.find_element_by_link_text('Comparativos').click()
driver.find_element_by_link_text('Quantitativos').click()
driver.find_element_by_link_text('Mecenato').click()
driver.find_element_by_link_text('De projetos apresentados').click()
driver.find_element_by_link_text('Por ano, região, uf, área e segmento cultural').click()
element1 = driver.find_element_by_xpath('//img[@alt="Ministério da Cultura"]')
hov1 = ActionChains(driver).move_to_element(element1)
hov1.perform()

frame = driver.find_element_by_name('Salicnet_iframe')
driver.switch_to.frame(frame)

wait = WebDriverWait(driver, 60)
wait.until(EC.visibility_of_element_located((By.XPATH, '//input[@name="sub_form"]')))

driver.execute_script('document.getElementById("sub_formb").click()')

In [ ]:
#Pegar a tabela que contém os dados desejados
html=driver.page_source
soup=BeautifulSoup(html,'html.parser')
tables=soup.findAll("table")
driver.switch_to.default_content()

In [ ]:
table=pd.read_html(str(tables[5]), header = 0)

In [ ]:
#dataframe da tabela com os dados
df = table[0]

#Criando uma couluna para identificar os dados de projetos já apresentados
df['Tipo'] = desc

#Substituindo todos os NaN por 0
df = df.fillna(0)
    